In [46]:
import pandas
import numpy as np
import sklearn
from tqdm import tqdm_notebook as tqdm
import csv
import os

In [47]:
#import libs
# Python 2 and 3 compatibility
# pip install future
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# будем отображать графики прямо в jupyter'e
%pylab inline

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
import seaborn as sns

#!!! if lib is not installed call "!pip install <libname>" in jupyter

Populating the interactive namespace from numpy and matplotlib


In [48]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go
import plotly.tools as tls
init_notebook_mode(connected=True)

In [49]:
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import cross_validate

In [50]:
#data = pandas.read_csv('deltasSubstraction1.csv',sep=";|,", engine="python")

In [51]:
#data = pandas.read_csv('deltasSubstractionCards.csv',sep=";|,", engine="python")

In [52]:
data = pandas.read_csv('deltasSubstractionCardsContracts.csv',sep=";|,", engine="python")

In [53]:
#data = data.drop(['REPORT_MONTH','CLIENT_HASH_ID','GENDER','F1','AGE'],1)

In [54]:
#data = data.drop(['REPORT_MONTH','CLIENT_HASH_ID','GENDER','F1'],1).dropna()

In [55]:
data = data.drop(['REPORT_MONTH','CLIENT_HASH_ID','F1'],1).dropna()
data['GENDER'] = data['GENDER'].map({'F': 1, 'M': 0})

In [56]:
active = data.loc[data['ACTIVE_FLAG']==1].sample(n=22031,random_state = 1)
nonactive = data.loc[data['ACTIVE_FLAG']==0]
dataset = pandas.concat([active,nonactive])

In [57]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score

In [39]:

clf = RandomForestClassifier(n_estimators=10)
scoring = {'acc': 'accuracy',
           'prec_macro': 'precision_macro',
           'rec_micro': 'recall_macro',
          'f1':'f1_macro'}
scores = cross_validate(clf, dataset.drop(['ACTIVE_FLAG'],1), dataset['ACTIVE_FLAG'], scoring=scoring,
                        cv=2, return_train_score=True)
sorted(scores.keys())
scores  

{'fit_time': array([ 5.83400011,  5.75999999]),
 'score_time': array([ 3.37100005,  3.37199998]),
 u'test_acc': array([ 0.96021357,  0.96006238]),
 u'test_f1': array([ 0.49968127,  0.50097326]),
 u'test_prec_macro': array([ 0.53644978,  0.54000585]),
 u'test_rec_micro': array([ 0.50371795,  0.50438048]),
 u'train_acc': array([ 0.99570707,  0.99571046]),
 u'train_f1': array([ 0.96829502,  0.96829496]),
 u'train_prec_macro': array([ 0.99358814,  0.99407044]),
 u'train_rec_micro': array([ 0.94564651,  0.94526031])}

In [58]:
train = dataset.sample(n=22031,random_state = 7)
features = train.drop(['ACTIVE_FLAG'],1)
goals = train['ACTIVE_FLAG']
predict = dataset.sample(n=22031,random_state = 191)

In [59]:
#train = data
#features = train.drop(['ACTIVE_FLAG'],1)
#goals = train['ACTIVE_FLAG']
#predict = data

In [60]:
clf.fit(features, goals)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [61]:
truval = predict['ACTIVE_FLAG'].values
res = clf.predict(predict.drop(['ACTIVE_FLAG'],1))
tp = 0
fp = 0
tn = 0
fn = 0
for goal, predicted in zip(truval,res):
    if bool(goal):
        if predicted == 1:           
            tp += 1
            continue
        fn += 1
        continue
    if bool(predicted):
        fp += 1
        continue
    tn += 1

        
print(tp)
print(tn)
print(fp)
print(fn)
print("Полнота уходящих")
print(float(tn)/(tn+fp))

print("Полнота оставшихся")
print(float(tp)/(tp+fn))

print("Accuracy")
print(float(tp+tn)/(tp+fp+tn+fn))

prec = float(tp)/(tp+fp)
recall = float(tp)/(tp+fn)
print("Precision")
print(prec)
print("Recall")
print(recall)
print("Fmeasure")
print (2*prec*recall/(prec+recall))

8610
9397
1564
2460
Полнота уходящих
0.857312289025
Полнота оставшихся
0.777777777778
Accuracy
0.817348281966
Precision
0.846274818164
Recall
0.777777777778
Fmeasure
0.810581811335


In [325]:
predict.ACTIVE_FLAG.value_counts()

1    11070
0    10961
Name: ACTIVE_FLAG, dtype: int64

In [326]:
zip(list(dataset)[1:],clf.feature_importances_)

[('GOOD_PHONE', 0.018599332683851128),
 ('GOOD_EMAIL', 0.017142313248533644),
 ('GENDER', 0.019550186769510022),
 ('AGE', 0.12617725390038159),
 ('TRN_CNT', 0.071122482696463679),
 ('TRN_CNT_POS', 0.062763147506172595),
 ('TRN_CNT_ATM', 0.052502428025981339),
 ('TRN_AMNT', 0.10658435219978322),
 ('TRN_AMNT_POS', 0.09795389815190006),
 ('TRN_AMNT_ATM', 0.10436167191022189),
 ('CONTRACT_TURN_AMOUNT', 0.10707294503952594),
 ('SALARY_INCOME', 0.057186522560675843),
 ('DBO_AUTH_WWW', 0.017601193290687912),
 ('DBO_AUTH_MB', 0.011431479119154042),
 ('DBO_OPER_CNT', 0.009599449417897199),
 ('DBO_REM_CNT', 0.0044888361873776008),
 ('DBO_PAY_CNT', 0.0082888280437831609),
 ('DBO_OPER_AMNT', 0.012999738642097591),
 ('DBO_REM_AMNT', 0.0070654908799533782),
 ('DBO_PAY_AMNT', 0.011122299864736486),
 ('CARD_COUNT', 0.036573875672869093),
 ('CONTRACT_COUNT', 0.039812274188442631)]

In [313]:
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
res = gnb.fit(features, goals).predict(predict.drop(['ACTIVE_FLAG'],1))
tp = 0
fp = 0
tn = 0
fn = 0
for goal, predicted in zip(truval,res):
    if bool(goal):
        if predicted == 1:           
            tp += 1
            continue
        fn += 1
        continue
    if bool(predicted):
        fp += 1
        continue
    tn += 1

        
print(tp)
print(tn)
print(fp)
print(fn)
print("Полнота уходящих")
print(float(tn)/(tn+fp))

print("Полнота оставшихся")
print(float(tp)/(tp+fn))

print("Accuracy")
print(float(tp+tn)/(tp+fp+tn+fn))

prec = float(tp)/(tp+fp)
recall = float(tp)/(tp+fn)
print("Precision")
print(prec)
print("Recall")
print(recall)
print("Fmeasure")
print (2*prec*recall/(prec+recall))

1137
10520
441
9933
Полнота уходящих
0.959766444667
Полнота оставшихся
0.1027100271
Accuracy
0.529118060914
Precision
0.720532319392
Recall
0.1027100271
Fmeasure
0.179791271347


In [62]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)

In [63]:
clf.fit(features, goals)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [64]:
truval = predict['ACTIVE_FLAG'].values
r = clf.predict(predict.drop(['ACTIVE_FLAG'],1))
res=[]
for i in r:
    if i>0.5:
        res.append(1)
        continue
    res.append(0)
tp = 0
fp = 0
tn = 0
fn = 0
for goal, predicted in zip(truval,res):
    if bool(goal):
        if predicted == 1:           
            tp += 1
            continue
        fn += 1
        continue
    if bool(predicted):
        fp += 1
        continue
    tn += 1

        
print(tp)
print(tn)
print(fp)
print(fn)
print("Полнота уходящих")
print(float(tn)/(tn+fp))

print("Полнота оставшихся")
print(float(tp)/(tp+fn))

print("Accuracy")
print(float(tp+tn)/(tp+fp+tn+fn))

prec = float(tp)/(tp+fp)
recall = float(tp)/(tp+fn)
print("Precision")
print(prec)
print("Recall")
print(recall)
print("Fmeasure")
print (2*prec*recall/(prec+recall))

6946
6675
4286
4124
Полнота уходящих
0.608977283095
Полнота оставшихся
0.627461607949
Accuracy
0.618265171803
Precision
0.618411680912
Recall
0.627461607949
Fmeasure
0.622903775446


In [212]:
dat = pandas.read_csv('deltasSubstractionCardsContracts.csv',sep=";|,", engine="python")
dat = dat.drop(['REPORT_MONTH','CLIENT_HASH_ID','GENDER'],1).dropna()

In [224]:
results=[]

for clas in tqdm(unique(dat.F1)):
    data = dat.loc[dat.F1 == clas].drop(['F1'],1)
    nonactive = data.loc[data['ACTIVE_FLAG']==0]
    active = data.loc[data['ACTIVE_FLAG']==1].sample(n=len(nonactive),random_state = 1)
    dataset = pandas.concat([active,nonactive])
    
    if(len(dataset) <2):
        continue
    
    train = dataset.sample(n=len(dataset)/2,random_state = 7)
    features = train.drop(['ACTIVE_FLAG'],1)
    goals = train['ACTIVE_FLAG']
    predict = dataset.sample(n=len(dataset)/2,random_state = 191)
    
    clf = RandomForestClassifier(n_estimators=10)
    clf.fit(features, goals)
    truval = predict['ACTIVE_FLAG'].values
    res = clf.predict(predict.drop(['ACTIVE_FLAG'],1))
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for goal, predicted in zip(truval,res):
        if bool(goal):
            if predicted == 1:           
                tp += 1
                continue
            fn += 1
            continue
        if bool(predicted):
            fp += 1
            continue
        tn += 1
    if (tp+fp==0):
        prec = 0
    else:
        prec = float(tp)/(tp+fp)
    if (tp+fn==0):
        recall = 0
    else:
        recall = float(tp)/(tp+fn)    
    if (tn+fp==0):
        polnuh=0
    else:
        polnuh=float(tn)/(tn+fp)
    
    if (tp+fn==0):
        polnost=0
    else:
        polnost=float(tp)/(tp+fn)
    
    if (tp+tn==0):
        acc=0
    else:
        acc=(float(tp+tn)/(tp+fp+tn+fn))
       
    if (prec+recall==0):
        fm=0
    else:
        fm=2*prec*recall/(prec+recall)
    
    results.append([clas,"forest",polnuh,polnost,acc,prec,recall,fm])

    
    gnb = GaussianNB()
    res = gnb.fit(features, goals).predict(predict.drop(['ACTIVE_FLAG'],1))
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for goal, predicted in zip(truval,res):
        if bool(goal):
            if predicted == 1:           
                tp += 1
                continue
            fn += 1
            continue
        if bool(predicted):
            fp += 1
            continue
        tn += 1
    if (tp+fp==0):
        prec = 0
    else:
        prec = float(tp)/(tp+fp)
    if (tp+fn==0):
        recall = 0
    else:
        recall = float(tp)/(tp+fn)    
    if (tn+fp==0):
        polnuh=0
    else:
        polnuh=float(tn)/(tn+fp)
    
    if (tp+fn==0):
        polnost=0
    else:
        polnost=float(tp)/(tp+fn)
    
    if (tp+tn==0):
        acc=0
    else:
        acc=(float(tp+tn)/(tp+fp+tn+fn))
       
    if (prec+recall==0):
        fm=0
    else:
        fm=2*prec*recall/(prec+recall)
    
    results.append([clas,"naive",polnuh,polnost,acc,prec,recall,fm])

    cls = sklearn.linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1) 
    clf.fit(features, goals)
    
    truval = predict['ACTIVE_FLAG'].values
    r = clf.predict(predict.drop(['ACTIVE_FLAG'],1))
    res=[]
    for i in r:
        if i>0.5:
            res.append(1)
            continue
        res.append(0)
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for goal, predicted in zip(truval,res):
        if bool(goal):
            if predicted == 1:           
                tp += 1
                continue
            fn += 1
            continue
        if bool(predicted):
            fp += 1
            continue
        tn += 1
    if (tp+fp==0):
        prec = 0
    else:
        prec = float(tp)/(tp+fp)
    if (tp+fn==0):
        recall = 0
    else:
        recall = float(tp)/(tp+fn)    
    if (tn+fp==0):
        polnuh=0
    else:
        polnuh=float(tn)/(tn+fp)
    
    if (tp+fn==0):
        polnost=0
    else:
        polnost=float(tp)/(tp+fn)
    
    if (tp+tn==0):
        acc=0
    else:
        acc=(float(tp+tn)/(tp+fp+tn+fn))
       
    if (prec+recall==0):
        fm=0
    else:
        fm=2*prec*recall/(prec+recall)
    
    results.append([clas,"regression",polnuh,polnost,acc,prec,recall,fm])



In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.datasets import make_classification

In [19]:
names = ["Nearest Neighbors", "RBF SVM",
         "Decision Tree", "Random Forest", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)

truval = predict['ACTIVE_FLAG'].values

# iterate over classifiers
for name, clf in zip(names, classifiers):
    clf.fit(features, goals)
    res = clf.predict(predict.drop(['ACTIVE_FLAG'],1))
    print(name)
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for goal, predicted in zip(truval,res):
        if bool(goal):
            if predicted == 1:           
                tp += 1
                continue
            fn += 1
            continue
        if bool(predicted):
            fp += 1
            continue
        tn += 1
            
    print(tp)
    print(tn)
    print(fp)
    print(fn)
    print("Полнота уходящих")
    print(float(tn)/(tn+fp))
    
    print("Полнота оставшихся")
    print(float(tp)/(tp+fn))
    
    print("Accuracy")
    print(float(tp+tn)/(tp+fp+tn+fn))
    
    prec = float(tp)/(tp+fp)
    recall = float(tp)/(tp+fn)
    print("Precision")
    print(prec)
    print("Recall")
    print(recall)
    print("Fmeasure")
    print (2*prec*recall/(prec+recall))
    print(" ")
    print(" ")

Nearest Neighbors
7383
7946
3015
3687
Полнота уходящих
0.7249338564
Полнота оставшихся
0.666937669377
Accuracy
0.695792292678
Precision
0.710040392383
Recall
0.666937669377
Fmeasure
0.687814421465
 
 
RBF SVM
5576
10871
90
5494
Полнота уходящих
0.99178907034
Полнота оставшихся
0.503703703704
Accuracy
0.746538967818
Precision
0.984115778327
Recall
0.503703703704
Fmeasure
0.666347992352
 
 
Decision Tree
6685
7858
3103
4385
Полнота уходящих
0.716905391844
Полнота оставшихся
0.603884372177
Accuracy
0.660115292088
Precision
0.682979158153
Recall
0.603884372177
Fmeasure
0.641001054751
 
 
Random Forest
6337
7700
3261
4733
Полнота уходящих
0.702490648663
Полнота оставшихся
0.572448057814
Accuracy
0.637147655576
Precision
0.660241717024
Recall
0.572448057814
Fmeasure
0.613218502032
 
 
AdaBoost
6779
7903
3058
4291
Полнота уходящих
0.721010856674
Полнота оставшихся
0.612375790425
Accuracy
0.666424583541
Precision
0.689132865711
Recall
0.612375790425
Fmeasure
0.64849093605
 
 
Naive Bayes
1137
